In [1]:
# !pip install transformers torch faiss-cpu pdfplumber sentence-transformers


  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached pillow-11.0.0-cp312-cp312-win_amd64.whl.metadata (9.3 kB)
  Using cached scikit_learn-1.6.0-cp312-cp312-win_amd64.whl.metadata (15 kB)
  Using cached scipy-1.14.1-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/10.1 MB ? eta -:--:--
   --------------------- ------------------ 5.5/10.1 MB 30.5 MB/s eta 0:00:01
   -------------------------------------- - 9.7/10.1 MB 24.1 MB/s eta 0:00:01
   ---------------------------------------- 10.1/10.1 MB 23.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/203.0 MB ? eta -:--:--
    --------------------------------------- 4.7/203.0 MB 22.0 MB/s eta 0:00:10
   - -------------------------------------- 8.7/203.0 MB 20.6 MB/s eta 0:00:10
   -- ------------------------------------- 12.8/2

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chromadb 0.5.23 requires tokenizers<=0.20.3,>=0.13.2, but you have tokenizers 0.21.0 which is incompatible.


In [ ]:
import pdfplumber
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
from transformers import pipeline

# Initialize ChromaDB client
def initialize_chromadb():
    client = chromadb.Client(Settings(
        chroma_db_impl="duckdb+parquet",
        persist_directory="./chromadb_store"  # Directory to persist the data
    ))
    collection = client.get_or_create_collection("pdf_embeddings")
    return client, collection

# Step 1: Extract text from PDF
def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

# Step 2: Split text into paragraphs
def split_into_paragraphs(text):
    return [para.strip() for para in text.split('\n') if para.strip()]

# Step 3: Embed and Store Paragraphs in ChromaDB
def store_paragraphs_in_chromadb(paragraphs, collection, model_name="sentence-transformers/all-mpnet-base-v2"):
    model = SentenceTransformer(model_name)
    embeddings = model.encode(paragraphs, convert_to_tensor=False)
    
    # Add to ChromaDB collection
    for idx, (para, embedding) in enumerate(zip(paragraphs, embeddings)):
        collection.add(
            documents=[para],
            metadatas=[{"id": idx}],
            embeddings=[embedding.tolist()]
        )
    print(f"Stored {len(paragraphs)} paragraphs in ChromaDB.")
    return model

# Step 4: Retrieve Relevant Paragraphs
def retrieve_relevant_paragraphs(query, collection, model, top_k=3):
    query_embedding = model.encode([query], convert_to_tensor=False).tolist()
    results = collection.query(query_embeddings=query_embedding, n_results=top_k)

    relevant_paragraphs = results["documents"][0]
    return relevant_paragraphs

# Step 5: Answer Questions Using Retrieved Paragraphs
def answer_question_with_rag(question, paragraphs):
    qa_pipeline = pipeline("question-answering", model="bert-large-uncased-whole-word-masking-finetuned-squad")
    best_answer = {"score": 0, "paragraph": "", "answer": ""}

    for para in paragraphs:
        try:
            result = qa_pipeline(question=question, context=para)
            if result['score'] > best_answer['score']:
                best_answer = {"score": result['score'], "paragraph": para, "answer": result['answer']}
        except Exception as e:
            print(f"Error processing paragraph: {e}")
            continue

    return best_answer

# Main Function
if __name__ == "__main__":
    pdf_path = "example.pdf"  # Replace with your PDF file path

    # Initialize ChromaDB
    print("\nInitializing ChromaDB...")
    chromadb_client, collection = initialize_chromadb()

    # Extract and process text
    print("\nExtracting text from PDF...")
    text = extract_text_from_pdf(pdf_path)
    paragraphs = split_into_paragraphs(text)

    # Store paragraphs in ChromaDB
    print("\nStoring paragraphs in ChromaDB...")
    embedding_model = store_paragraphs_in_chromadb(paragraphs, collection)

    # Retrieve relevant paragraphs

    question = input("Enter your question: ")

    print("\nRetrieving relevant paragraphs...")
    relevant_paragraphs = retrieve_relevant_paragraphs(question, collection, embedding_model)

    # Answer the question
    print("\nAnswering the question...")
    result = answer_question_with_rag(question, relevant_paragraphs)

    # Display the results
    if result['score'] > 0:
        print("\nAnswer:", result['answer'])
        print("\nRelevant Paragraph:", result['paragraph'])
    else:
        print("\nNo relevant answer found.")


In [3]:
import pdfplumber
from sentence_transformers import SentenceTransformer
import faiss
from transformers import pipeline

# Step 1: Extract text from PDF
def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

# Step 2: Split text into paragraphs
def split_into_paragraphs(text):
    return [para.strip() for para in text.split('\n') if para.strip()]

# Step 3: Create embeddings for paragraphs
def create_paragraph_embeddings(paragraphs, model_name="sentence-transformers/all-mpnet-base-v2"):
    model = SentenceTransformer(model_name)
    embeddings = model.encode(paragraphs, convert_to_tensor=False)
    return embeddings, model

# Step 4: Build a FAISS index
def build_faiss_index(embeddings):
    dim = len(embeddings[0])
    index = faiss.IndexFlatL2(dim)
    index.add(embeddings)
    return index

# Step 5: Retrieve relevant paragraphs
def retrieve_relevant_paragraphs(query, paragraphs, index, model, top_k=3):
    query_embedding = model.encode([query], convert_to_tensor=False)
    distances, indices = index.search(query_embedding, top_k)
    relevant_paragraphs = [paragraphs[i] for i in indices[0]]
    return relevant_paragraphs

# Step 6: Answer questions using retrieved paragraphs
def answer_question_with_rag(question, paragraphs):
    qa_pipeline = pipeline("question-answering", model="bert-large-uncased-whole-word-masking-finetuned-squad")
    best_answer = {"score": 0, "paragraph": "", "answer": ""}

    for para in paragraphs:
        try:
            result = qa_pipeline(question=question, context=para)
            if result['score'] > best_answer['score']:
                best_answer = {"score": result['score'], "paragraph": para, "answer": result['answer']}
        except Exception as e:
            print(f"Error processing paragraph: {e}")
            continue

    return best_answer

# Main Function
if __name__ == "__main__":
    pdf_path = "example1.pdf"  # Replace with your PDF file path
    question = input("Enter your question: ")

    # Extract and process text
    print("\nExtracting text from PDF...")
    text = extract_text_from_pdf(pdf_path)
    paragraphs = split_into_paragraphs(text)

    # Create embeddings and build FAISS index
    print("\nCreating embeddings and building FAISS index...")
    embeddings, embedding_model = create_paragraph_embeddings(paragraphs)
    faiss_index = build_faiss_index(embeddings)

    # Retrieve relevant paragraphs
    print("\nRetrieving relevant paragraphs...")
    relevant_paragraphs = retrieve_relevant_paragraphs(question, paragraphs, faiss_index, embedding_model)

    # Answer the question
    print("\nAnswering the question...")
    result = answer_question_with_rag(question, relevant_paragraphs)

    # Display the results
    if result['score'] > 0:
        print("\nAnswer:", result['answer'])
        print("\nRelevant Paragraph:", result['paragraph'])
    else:
        print("\nNo relevant answer found.")



Extracting text from PDF...

Creating embeddings and building FAISS index...

Retrieving relevant paragraphs...

Answering the question...


e:\anaconda3\envs\rag\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kesha\.cache\huggingface\hub\models--bert-large-uncased-whole-word-masking-finetuned-squad. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad 


Answer: Long Short-Term Memory

Relevant Paragraph: LSTM Long Short-Term Memory


In [4]:
# pdf_path = "example.pdf"  # Replace with your PDF file path
# question = input("Enter your question: ")

# # Extract and process text
# print("\nExtracting text from PDF...")
# text = extract_text_from_pdf(pdf_path)
# paragraphs = split_into_paragraphs(text)

# # Create embeddings and build FAISS index
# print("\nCreating embeddings and building FAISS index...")
# embeddings, embedding_model = create_paragraph_embeddings(paragraphs)
# faiss_index = build_faiss_index(embeddings)

# Retrieve relevant paragraphs
print("\nRetrieving relevant paragraphs...")
relevant_paragraphs = retrieve_relevant_paragraphs(question, paragraphs, faiss_index, embedding_model)

# Answer the question
print("\nAnswering the question...")
result = answer_question_with_rag(question, relevant_paragraphs)

# Display the results
if result['score'] > 0:
    print("\nAnswer:", result['answer'])
    print("\nRelevant Paragraph:", result['paragraph'])
else:
    print("\nNo relevant answer found.")


Extracting text from PDF...

Creating embeddings and building FAISS index...

Retrieving relevant paragraphs...

Answering the question...


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu



Answer: to improve performance

Relevant Paragraph: to improve performance, such as making the RNN bidirectional and/or stateful.
